In [ ]:
"""""""""""""""""
Required packages
"""""""""""""""""
import math
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt

# <font color=darkcyan>Modèles de mélange</font>

## <font color=darkorange>Densité de mélange </font>

Considérons un mélange de $K$ lois dont la densité est donnée par
$$f(x, \theta)=\sum_{k=1}^K\pi_kf(x, \theta_k),$$
où $0<\pi_k<1$, $\sum_{k=1}^K\pi_k=1$, $\theta = (\theta_1, \ldots, \theta_K)$. 

Dans l'optique du calcul de l'estimateur de maximum de vraisemblance, nous désirons calculer la logdensité du mélange, à partir des logdensités des composantes du mélange. 

#### Question 1
Ecrire une fonction `mixture_logpdf` qui prend en argument une liste de logdensités et une liste de poids et renvoie la logdensité du mélange associé.

In [ ]:
def mixture_logpdf(log_probs, weights):
    """
    Inputs
    ----------
    log_probs: loglikelihood of each term
    weights: weights of the components of the mixture
    
    Outputs
    -------
    logp: loglikelihood of the mixture
    """
    
    def logp(x):
        log_marginals = np.array([log_probs[j](x) for j in range(len(log_probs))])
        return np.log(np.einsum('i,i...', weights, np.exp(log_marginals)))
        #V2 : mais cela demande de faire un reshape sur l'entree, quand on évalue sur une grille
        #return np.log(np.dot(weights, np.exp(log_marginals)))
    
    return logp

### Mélange de gaussiennes multivariées

Considérons un mélange de $K$ lois gaussiennes multivariées dont la densité est donnée par
$$f_\theta(x)=\sum_{k=1}^K\pi_kf_{\mathcal N(\mu_k,\Sigma_k)}(x),$$
où $0<\pi_k<1$, $\sum_{k=1}^K\pi_k=1$, $\mu_k\in\mathbb R^d$ et $\Sigma_k \in \mathcal{M}_d(\mathbb R)$ semi-definie positive, $k=1,\dots,K$. 

#### Question 2
Ecrire une fonction `multi_gauss_logpdf` qui prend en argument une moyenne et une matrice de covariance et renvoie la logdensité gaussienne associée à ces paramètres. On écrira une fonction pour une variable en dimension $d>1$. On rappelle que

$$f_{\mathcal N(\mu,\Sigma)}(x) = \frac {1}{(2\pi )^{d/2} \det(\Sigma )^{1/2}}\;\;e^{-{\frac {1}{2}}(x-\mu )^{\top }\Sigma ^{-1}(x-\mu )} \quad \forall x\in\mathbb R ^d$$

$$\log f_{\mathcal N(\mu,\Sigma)}(x) = -\frac 1 2 \left(d\log(2\pi) +\log(\det(\Sigma)) + (x-\mu )^{\top }\Sigma ^{-1}(x-\mu ) \right)$$

In [ ]:
def multi_gauss_logpdf(mu, sigma):
    """
    Inputs
    ----------
    mu: mean of the Gaussian distribution
    sigma: covariance matrix of the Gaussian distribution
    
    Outputs
    -------
    logp: loglikelihood
    """

    def logp(x): 
        """
        Inputs
        ----------
        x: array of size nxd
    
        Outputs
        -------
        Gaussian logdensity evaluated on all n points
        """
        d = mu.shape[0]
        cst = d * np.log(2 * np.pi)
        det = np.log(np.linalg.det(sigma))
        diff = x-mu
        quad_term = np.einsum('...i,ij,...j', diff, np.linalg.inv(sigma), diff)
        #V2 : mais ca va demander de faire un reshape sur l'entree, quand on évalue sur une grille
        #first_prod = diff @ np.linalg.inv(sigma)
        #quad_term = np.sum(first_prod*diff,axis=-1) 
        
        return - 0.5 *(cst +  det + quad_term) 
    
    return logp

#### Question 3
Utilisez la fonction `mixture_logpdf` pour tracer la densité d'un mélange de trois gaussiennes en dimension 2, de paramètres
\begin{equation*}
\mu_1 = \binom 2  2, 
\Sigma_1 = 
\begin{pmatrix}
1. & 0.5 \\
0.5 & 1.
\end{pmatrix}, \quad
\mu_2 = -\mu_1, 
\Sigma_2 = 
\begin{pmatrix}
1. & -0.1 \\
-0.1 & 1.
\end{pmatrix}, \quad
\mu_3 = \binom{-1.5 }{ 2.2}, 
\Sigma_3 = 
\begin{pmatrix}
0.8 & 0 \\
0 & 0.8
\end{pmatrix}
\end{equation*}

Les mélanges de lois gaussiennes peuvent être implémentés directement en Python en utilisant le package `sklearn.mixture`, voir ici https://scikit-learn.org/stable/modules/mixture.html pour une aide détaillée (non nécessaire pour cette question).


In [ ]:
#params 
mu1 = 2*np.ones(2)
cov1 = np.array([[1., 0.5],
                [0.5, 1.]])
mu2 = -mu1
cov2 = np.array([[1., -0.1],
                [-0.1, 1.]])

mu3 = np.array([-1.5, 2.2])
cov3 = 0.8 * np.eye(2)

In [ ]:
mixt  = mixture_logpdf([multi_gauss_logpdf(mu1, cov1), multi_gauss_logpdf(mu2, cov2), multi_gauss_logpdf(mu3, cov3)], [0.25, 0.35, 0.4])

plt.figure(figsize=(8, 8), dpi=80)

grid_lim = 6
# grid on which the target pdf is displayed
grid_plot = (-grid_lim, grid_lim, -grid_lim, grid_lim)
# coordinates chosen on this grid
nb_points = 100

xplot = np.linspace(-grid_lim, grid_lim, nb_points)
yplot = np.linspace(-grid_lim, grid_lim, nb_points)
X, Y = np.meshgrid(xplot, yplot)

#V2 : 
#XY = np.array([X.ravel(),Y.ravel()]).T

XY = np.dstack((X,Y))
print(XY.shape)

Zplot = np.exp(mixt(XY))
#V2 : 
#Zplot = Zplot.reshape(X.shape)

fig, ax = plt.subplots(figsize=(8,10))
ax.grid(True, color="white")
pcm = ax.pcolormesh(X, Y, Zplot, cmap = 'Blues')
fig.colorbar(pcm,orientation='horizontal')

In [ ]:
Zplot[0,0] = np.exp(mixt(np.array((X[0][0], Y[0][0]))))
Zplot[0,0]

## Mélange de lois uniformes
Considérons les mélanges de lois uniformes $U_k \sim \mathcal U([0,\lambda_k])$, $k=1,\ldots,K$, dont la densité est donnée par 
$$f_\theta(x)=\sum_{k=1}^K\pi_kf_{U_k}(x),$$
où $0<\pi_k<1$, $\sum_{k=1}^K\pi_k=1$ et $\lambda_k>0$, $k=1,\dots,K$.

#### Question 4
Écrire une fonction `uniform_logpdf` qui prend en argument un paramètre $\lambda$ et renvoie la logdensité de la loi uniforme $U\sim\mathcal U([0,\lambda])$. 

Quelles formes de densité peut-on obtenir avec un mélange de lois uniformes ?

Afficher dans un même graphe la densité du mélange uniforme de paramètres $(\lambda_1, \lambda_2, \lambda_3) = (2,7,15)$ et de poids $(w_1, w_2, w_3) = (0.15, 0.40,0.45)$, ainsi que les densités de chacune de ses composantes.


In [ ]:
def uniform_logpdf(lmbd):
    """
    Inputs
    ----------
    lmbd: upper bound of the support of the uniform distribution
    
    Outputs
    -------
    logpdf: loglikelihood
    """

    def logpdf(x):
        loglik = -np.log(lmbd)*np.ones(x.shape[0])
        outer = ~((x>0) * (x<lmbd))
        loglik[outer] = -math.inf
        return loglik
    
    return logpdf

In [ ]:
weights =  np.array([0.15, 0.40,0.45])
lambdas = np.array([2,7.5,15.5])  

In [ ]:
mixt_unif_logpdf  = mixture_logpdf([uniform_logpdf(lmbd) for lmbd in lambdas],weights)

nb_points = 100
x = np.linspace(0.01, 25.0, nb_points)

fig, ax = plt.subplots()

ax.plot(x,np.exp(mixt_unif_logpdf(x)),'b', lw=1, label = 'Mixture of uniform distributions')
for i,lmbd in enumerate(lambdas):
    ax.plot(x, np.exp(uniform_logpdf(lmbd)(x)), 'b--', alpha = 0.5, lw=1, label = fr'Component $\lambda=${lmbd}, $\pi=${weights[i]}')
ax.tick_params(labelright=True)
ax.grid(True)
ax.legend()

## Mélange de lois Gamma

Considérons les mélanges de loi Gamma $\Gamma(\alpha,\beta)$ dont la densité est donnée par 
$$f_\theta(x)=\sum_{k=1}^K\pi_kf_{\Gamma(\alpha_k,\beta_k)}(x),$$
où $0<\pi_k<1$, $\sum_{k=1}^K\pi_k=1$ et $\alpha_k>0, \beta_k>0$ pour $k=1,\dots,K$. 

#### Question 5
Écrire une fonction `gamma_logpdf` qui prend en argument deux paramètres $a$ et $b$ et renvoie la logdensité de la loi $\Gamma(a,b)$. 

Quelles formes de densité peut-on obtenir avec un mélange de lois Gamma ? 


Afficher dans un même graphe la densité du mélange de lois Gamma de paramètres $a = (2.0,4.0,30.0)$ et $
b = (1.0,0.5,2.5)$ et de poids $(w_1, w_2, w_3) = (0.15, 0.40,0.45)$, ainsi que les densités de chacune de ses composantes.

Comparer aux formes des mélanges gaussiens.

In [ ]:
def gamma_logpdf(a,b):
    """
    Inputs
    ----------
    a, b: shapes and inverse scales of the gamma distribution
    
    Outputs
    -------
    logp: loglikelihood
    """

    def logp(x):
        loglik = a*np.log(b) 
        loglik = loglik - np.log(math.gamma(a)) + (a-1)*np.log(x) - b*x
        return loglik
    
    return logp

In [ ]:
a = [2.0,4.0,30.0]
b = [1.0,0.5,2.5]

In [ ]:
mixt_gamma_logpdf = mixture_logpdf([gamma_logpdf(a_k,b_k) for a_k,b_k in zip(a,b)],weights)

fig, ax = plt.subplots()

ax.plot(x,np.exp(mixt_gamma_logpdf(x)),'b', lw=1, label = 'Mixture of gamma distributions')
for i,(a_k, b_k) in enumerate(zip(a,b)):
    ax.plot(x, np.exp(gamma_logpdf(a_k,b_k)(x)), 'b--', alpha = 0.5, lw=1, label = fr'Component $a=${a_k}, $b=${b_k}, $\pi=${weights[i]}')
ax.tick_params(labelright=True)
ax.grid(True)
ax.legend()

## <font color=darkorange>Simulation de données</font>

#### Question 5
- Ecrire une fonction `gaussian_mixture` pour générer des réalisations d'un mélange gaussien à $K$ composantes. Les arguments de la fonction sont la taille d'échantillon `n`, le moyennes `mu`, variances `sigma` et poids des éléments du mélange `weights`.
- Tester votre fonction en générant un grand échantillon d'un mélange gaussien de taille `n=5000` dont on compare l'histogramme à sa densité par un graphique.

In [ ]:
def gaussian_mixture(n, mu, sigma, weights):
    """
    Inputs
    ----------
    n: sample size
    mu: mean of each component
    sigma: std of each component
    weights: weights of the mixture
    
    Outputs
    -------
    samples: samples from the mixture model
    """
    
    x = np.random.normal(0, 1, n)
    I = np.random.choice(mu.shape[0], size=n, p=weights)
    
    return mu[I]+sigma[I]*x

In [ ]:
# number of observations
n = 5000

# means and variance to be estimated
mu = np.array([-2,3])
sigma = np.array([1.5,1])


# prior probability of the first cluster/goup 
weights = np.array([0.3,0.7])

samples = gaussian_mixture(n, mu, sigma, weights)

In [ ]:
def Gaussian_pdf(x, mean, variance):
  z = np.exp(-(x - mean)**2/(2*variance))/np.sqrt(2*np.pi*variance)
  return z

count, bins, ignored = plt.hist(samples, 100, density=True)
x = np.linspace(np.min(mu) - 3*np.max(sigma),
                np.max(mu) + 3*np.max(sigma), 100)
plt.plot(x, Gaussian_pdf(x, mu[0], sigma[0]), linestyle = 'dashed', alpha = 0.5, color='red', label="True 1st density")
plt.plot(x, Gaussian_pdf(x, mu[1], sigma[1]), linestyle = 'dashed', alpha = 0.5, color='k', label="True 2nd density")
plt.plot(x, weights[0]*Gaussian_pdf(x, mu[0], sigma[0]) + (1-weights[0])*Gaussian_pdf(x, mu[1], sigma[1]), alpha = 0.5, color='b', label="True mixture density")

plt.title('Mixture of Gaussian distributions')
plt.tick_params(labelright=True)
plt.grid(True)
plt.legend();

## <font color=darkorange>Estimation du modèle</font>

#### Question 6
- Estimez les paramètres du modèle en utilisant les échantillons précédents et le package `mixture` de `sklearn`. On pourra utiliser la classe `GaussianMixture` et la méthode `fit`.
- Fournissez les paramètres estimés (à l'aide des attributs `means_`, `weights_`, etc.).

In [ ]:
from sklearn import mixture
clf = mixture.GaussianMixture(n_components=2)
X_train = np.reshape(samples, (5000,1))
clf.fit(X_train)

In [ ]:
clf.means_

In [ ]:
clf.weights_

In [ ]:
clf.covariances_